In [ ]:
import cv2
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib import font_manager
from IPython.display import Video, display, HTML, Image as IPImage
import tempfile
from ultralytics import YOLO
from typing import List, Dict, Tuple
import warnings
from tqdm import tqdm
import platform
warnings.filterwarnings('ignore')

In [ ]:
def setup_chinese_font():
    system = platform.system()

    chinese_fonts = []

    if system == 'Darwin':  # macOS
        chinese_fonts = [
            '/System/Library/Fonts/PingFang.ttc',
            '/System/Library/Fonts/Hiragino Sans GB.ttc',
        ]
    elif system == 'Windows':
        chinese_fonts = [
            'C:/Windows/Fonts/msyh.ttc',  # 微软雅黑
            'C:/Windows/Fonts/simhei.ttf',  # 黑体
            'C:/Windows/Fonts/simsun.ttc',  # 宋体
        ]
    else:  # Linux
        chinese_fonts = [
            '/usr/share/fonts/truetype/wqy/wqy-microhei.ttc',
            '/usr/share/fonts/truetype/arphic/uming.ttc',
        ]

    font_path = None
    for font in chinese_fonts:
        if Path(font).exists():
            font_path = font
            break

    if font_path:
        try:
            font_prop = font_manager.FontProperties(fname=font_path)
            plt.rcParams['font.family'] = font_prop.get_name()
            print(f"中文字体加载成功: {font_path}")
            return font_path
        except:
            pass

    print("⚠️ 未找到中文字体，使用英文标注")
    return None

CHINESE_FONT_PATH = setup_chinese_font()
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
def get_chinese_font_for_cv2(font_path: str = None, size: int = 20):

    from PIL import ImageFont

    if font_path is None:
        font_path = CHINESE_FONT_PATH

    if font_path and Path(font_path).exists():
        try:
            return ImageFont.truetype(font_path, size)
        except:
            pass

    return None

In [ ]:
def put_chinese_text(img, text, position, font_size=20, color=(255, 255, 255)):
    """
    在OpenCV图像上绘制中文文本（带黑色描边，无背景）

    Args:
        img: OpenCV图像
        text: 文本内容
        position: (x, y) 位置
        font_size: 字体大小
        color: 文字颜色 (B, G, R)
    """
    from PIL import Image, ImageDraw, ImageFont

    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)

    font = get_chinese_font_for_cv2(size=font_size)
    if font is None:
        x, y = position
        cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_SIMPLEX,
                   font_size/30, (0, 0, 0), 3, cv2.LINE_AA)  # 黑色描边
        cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_SIMPLEX,
                   font_size/30, color, 2, cv2.LINE_AA)  # 彩色文字
        return img

    x, y = position

    # PIL颜色转换 (BGR -> RGB)
    pil_color = (color[2], color[1], color[0])
    outline_color = (0, 0, 0)  # 黑色描边

    # 绘制描边（8个方向）
    outline_width = max(2, font_size // 10)
    for dx in [-outline_width, 0, outline_width]:
        for dy in [-outline_width, 0, outline_width]:
            if dx == 0 and dy == 0:
                continue
            draw.text((x + dx, y + dy), text, font=font, fill=outline_color)

    # 绘制主文字（彩色）
    draw.text((x, y), text, font=font, fill=pil_color)

    # 转回OpenCV
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

In [ ]:
class StreetObstacleDetector:

    def __init__(self,
                 model_path: str = "yolo11x.pt",
                 confidence: float = 0.3,
                 label_font_size: int = 18,
                 box_thickness: int = 2):
        """
        初始化检测器

        Args:
            model_path: YOLO模型路径
            confidence: 置信度阈值
            label_font_size: 标签字体大小
            box_thickness: 边框粗细
        """
        self.model = YOLO(model_path)
        self.confidence = confidence
        self.label_font_size = label_font_size
        self.box_thickness = box_thickness

        # 障碍物类别映射 (COCO数据集)
        self.obstacle_classes = {
            0: ("行人", "Person"),
            1: ("自行车", "Bicycle"),
            2: ("汽车", "Car"),
            3: ("摩托车", "Motorcycle"),
            5: ("公交车", "Bus"),
            7: ("卡车", "Truck"),
            9: ("红绿灯", "Traffic Light"),
            11: ("停止标志", "Stop Sign"),
            12: ("停车计时器", "Parking Meter"),
        }

        # 危险等级颜色 (BGR格式)
        self.colors = {
            "high": (0, 0, 255),      # 红色
            "medium": (0, 165, 255),   # 橙色
            "low": (0, 255, 0),        # 绿色
            "info": (255, 255, 0)      # 青色
        }

        print(f"YOLO模型加载成功: {model_path}")

    def _calculate_distance(self, bbox: List[float], img_height: int, img_width: int) -> float:

        x1, y1, x2, y2 = bbox

        # 计算框的面积占比
        box_area = (x2 - x1) * (y2 - y1)
        img_area = img_height * img_width
        area_ratio = box_area / img_area

        # 计算框底部位置（归一化到0-1）
        bottom_ratio = y2 / img_height

        # 计算框高度占比
        height_ratio = (y2 - y1) / img_height

        # 综合估算距离
        if area_ratio > 0.3:
            distance = 1.0
        elif area_ratio > 0.2:
            distance = 2.0
        elif area_ratio > 0.1:
            distance = 3.5
        elif area_ratio > 0.05:
            distance = 6.0
        elif area_ratio > 0.02:
            distance = 10.0
        else:
            if bottom_ratio > 0.9:
                distance = 8.0
            elif bottom_ratio > 0.7:
                distance = 15.0
            elif bottom_ratio > 0.5:
                distance = 25.0
            else:
                distance = 40.0

        # 根据高度占比微调
        if height_ratio > 0.6:
            distance *= 0.6
        elif height_ratio > 0.4:
            distance *= 0.8

        return round(distance, 1)

    def _get_direction(self, bbox: List[float], img_width: int) -> Tuple[str, str]:

        x1, y1, x2, y2 = bbox
        center_x = (x1 + x2) / 2

        if center_x < img_width * 0.33:
            return "左前方", "Left"
        elif center_x > img_width * 0.67:
            return "右前方", "Right"
        else:
            return "正前方", "Front"

    def _get_danger_level(self, class_name: str, distance: float, area_ratio: float) -> str:

        if class_name == "红绿灯":
            return "info"

        # 行人/自行车/摩托车
        if class_name in ["行人", "自行车", "摩托车"]:
            if distance < 3:
                return "high"
            elif distance < 6:
                return "medium"
            else:
                return "low"

        # 汽车/卡车/公交车
        elif class_name in ["汽车", "卡车", "公交车"]:
            if distance < 2:
                return "high"
            elif distance < 5:
                return "medium"
            else:
                return "low"

        # 其他障碍物
        else:
            if distance < 4:
                return "medium"
            else:
                return "low"

    def _find_label_position(self, bbox: List[float], occupied_regions: List,
                            img_height: int, img_width: int) -> Tuple[int, int]:
        """
        智能查找标签位置，避免重叠

        Args:
            bbox: 当前检测框
            occupied_regions: 已被占用的区域列表 [(x, y, w, h), ...]
            img_height: 图像高度
            img_width: 图像宽度

        Returns:
            (x, y) 标签左上角位置
        """
        x1, y1, x2, y2 = bbox
        label_width = 200
        label_height = 30

        candidates = [
            (int(x1), max(0, int(y1) - label_height - 5)),  # 上方
            (int(x1), int(y2) + 5),                          # 下方
            (max(0, int(x1) - label_width - 5), int(y1)),   # 左侧
            (int(x2) + 5, int(y1)),                          # 右侧
            (int(x1) + 5, int(y1) + 5),                      # 框内左上角
        ]

        for x, y in candidates:
            # 边界检查
            if x < 0 or y < 0 or x + label_width > img_width or y + label_height > img_height:
                continue

            overlap = False
            for ox, oy, ow, oh in occupied_regions:
                if not (x + label_width < ox or x > ox + ow or
                       y + label_height < oy or y > oy + oh):
                    overlap = True
                    break

            if not overlap:
                return (x, y)

        return (int(x1), max(0, int(y1) - label_height - 5))

    def detect_frame(self, frame: np.ndarray, use_chinese: bool = True) -> Tuple[np.ndarray, List[Dict]]:
        """
        检测单帧图像（优化标注布局）

        Args:
            frame: 输入图像
            use_chinese: 是否使用中文标注

        Returns:
            (标注后的图像, 检测结果列表)
        """
        img_height, img_width = frame.shape[:2]

        results = self.model(frame, conf=self.confidence, verbose=False)[0]

        annotated_frame = frame.copy()
        detections = []
        occupied_regions = []

        if hasattr(results, 'boxes') and len(results.boxes) > 0:
            for det in results.boxes.data:
                x1, y1, x2, y2, conf, cls = det.cpu().numpy()
                class_id = int(cls)

                if class_id not in self.obstacle_classes:
                    continue

                class_name_cn, class_name_en = self.obstacle_classes[class_id]
                confidence = float(conf)
                bbox = [float(x1), float(y1), float(x2), float(y2)]

                box_area = (x2 - x1) * (y2 - y1)
                img_area = img_height * img_width
                area_ratio = box_area / img_area

                distance = self._calculate_distance(bbox, img_height, img_width)
                direction_cn, direction_en = self._get_direction(bbox, img_width)
                danger_level = self._get_danger_level(class_name_cn, distance, area_ratio)

                detections.append({
                    "class_cn": class_name_cn,
                    "class_en": class_name_en,
                    "confidence": confidence,
                    "bbox": bbox,
                    "distance": distance,
                    "direction_cn": direction_cn,
                    "direction_en": direction_en,
                    "danger_level": danger_level,
                    "area_ratio": area_ratio
                })

                color = self.colors[danger_level]
                x1_int, y1_int, x2_int, y2_int = map(int, bbox)

                cv2.rectangle(annotated_frame, (x1_int, y1_int), (x2_int, y2_int),
                             color, self.box_thickness)

                label_x, label_y = self._find_label_position(
                    bbox, occupied_regions, img_height, img_width
                )

                if use_chinese and CHINESE_FONT_PATH:

                    label = f"{class_name_cn} {confidence:.2f} {direction_cn} {distance}m"

                    annotated_frame = put_chinese_text(
                        annotated_frame, label, (label_x, label_y),
                        font_size=self.label_font_size,
                        color=color
                    )

                    occupied_regions.append((label_x, label_y, 200, 30))

                else:
                    label = f"{class_name_en} {confidence:.2f} {direction_en} {distance}m"

                    font = cv2.FONT_HERSHEY_SIMPLEX
                    font_scale = 0.5
                    thickness = 2

                    (text_w, text_h), _ = cv2.getTextSize(label, font, font_scale, thickness)

                    cv2.putText(annotated_frame, label,
                               (label_x, label_y + text_h),
                               font, font_scale, (0, 0, 0), thickness + 2, cv2.LINE_AA)

                    cv2.putText(annotated_frame, label,
                               (label_x, label_y + text_h),
                               font, font_scale, color, thickness, cv2.LINE_AA)

                    occupied_regions.append((label_x, label_y, text_w + 10, text_h + 10))

        return annotated_frame, detections

In [ ]:
def test_distance_calculation():

    detector = StreetObstacleDetector()

    img_height, img_width = 960, 544

    test_cases = [
        ([100, 500, 500, 900], "大框，占据画面很大部分，靠近底部"),
        ([150, 600, 400, 850], "中等框，靠近底部"),
        ([200, 400, 350, 600], "中等框，中间位置"),
        ([250, 200, 350, 350], "小框，靠上位置"),
        ([100, 100, 200, 200], "小框，顶部位置"),
    ]

    print("距离估算测试：\n")
    for bbox, desc in test_cases:
        distance = detector._calculate_distance(bbox, img_height, img_width)
        x1, y1, x2, y2 = bbox
        area = (x2-x1) * (y2-y1)
        area_ratio = area / (img_height * img_width)
        print(f"{desc}")
        print(f"  框大小: {x2-x1}x{y2-y1} (面积占比: {area_ratio*100:.1f}%)")
        print(f"  估算距离: {distance}m\n")

In [ ]:
def test_random_frame(video_path: str,
                      model_path: str = "yolo11x.pt",
                      confidence: float = 0.3,
                      use_chinese: bool = True):
    """
    随机测试一帧，快速查看效果

    Args:
        video_path: 视频路径
        model_path: YOLO模型路径
        confidence: 置信度阈值
        use_chinese: 是否使用中文标注
    """
    print("单帧测试模式")

    detector = StreetObstacleDetector(model_path, confidence)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"无法打开视频: {video_path}")

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    random_frame_idx = np.random.randint(0, total_frames)
    cap.set(cv2.CAP_PROP_POS_FRAMES, random_frame_idx)

    ret, frame = cap.read()
    cap.release()

    if not ret:
        raise ValueError("无法读取帧")

    print(f"随机帧: {random_frame_idx}/{total_frames}")

    annotated_frame, detections = detector.detect_frame(frame, use_chinese)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

    ax1.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    ax1.set_title('原始图像', fontsize=16)
    ax1.axis('off')

    ax2.imshow(cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB))
    ax2.set_title(f'检测结果 (检测到 {len(detections)} 个对象)', fontsize=16)
    ax2.axis('off')

    plt.tight_layout()
    plt.show()

    if detections:
        print(f"\n检测详情:")
        for i, det in enumerate(detections, 1):
            print(f"  {i}. {det['class_cn']} ({det['class_en']})")
            print(f"     位置: {det['direction_cn']} ({det['direction_en']})")
            print(f"     距离: {det['distance']}m")
            print(f"     置信度: {det['confidence']:.2f}")
            print(f"     危险等级: {det['danger_level']}")
    else:
        print("未检测到障碍物")

    return annotated_frame, detections

In [ ]:
def process_video_with_progress(input_path: str,
                                output_path: str = None,
                                model_path: str = "yolo11x.pt",
                                confidence: float = 0.3,
                                use_chinese: bool = True,
                                show_info: bool = True,
                                save_detection_log: bool = True) -> str:
    """
    处理视频并生成检测结果 (带进度条)

    Args:
        input_path: 输入视频路径
        output_path: 输出视频路径
        model_path: YOLO模型路径
        confidence: 置信度阈值
        use_chinese: 是否使用中文标注
        show_info: 是否在视频上显示帧信息
        save_detection_log: 是否保存检测日志

    Returns:
        输出视频路径
    """
    print("完整视频处理模式")

    if not Path(input_path).exists():
        raise FileNotFoundError(f"找不到视频文件: {input_path}")

    if output_path is None:
        input_path_obj = Path(input_path)
        output_path = str(input_path_obj.parent / f"{input_path_obj.stem}_detected{input_path_obj.suffix}")

    detector = StreetObstacleDetector(model_path, confidence)

    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        raise ValueError(f"无法打开视频: {input_path}")

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"视频信息:")
    print(f"   分辨率: {width}x{height}")
    print(f"   帧率: {fps} fps")
    print(f"   总帧数: {total_frames}")
    print(f"   时长: {total_frames/fps:.1f}秒")

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    detection_log = []

    print("\n开始处理...")

    frame_count = 0

    pbar = tqdm(total=total_frames, desc="处理进度", unit="帧", ncols=100)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        annotated_frame, detections = detector.detect_frame(frame, use_chinese)

        if show_info:
            info_text = f"Frame: {frame_count}/{total_frames} | Objects: {len(detections)}"
            cv2.putText(annotated_frame, info_text,
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                       0.7, (0, 255, 255), 2)

        out.write(annotated_frame)

        if save_detection_log and detections:
            detection_log.append({
                "frame": frame_count,
                "timestamp": frame_count / fps,
                "detections": detections
            })

        pbar.update(1)
        pbar.set_postfix({"检测对象": len(detections)})

    pbar.close()

    cap.release()
    out.release()

    print(f"\n处理完成!")
    print(f"输出文件: {output_path}")

    if save_detection_log and detection_log:
        log_path = Path(output_path).with_suffix('.txt')
        with open(log_path, 'w', encoding='utf-8') as f:
            f.write("街景视频障碍物检测日志\n")
            f.write("=" * 80 + "\n\n")

            for log in detection_log:
                f.write(f"帧 {log['frame']} (时间: {log['timestamp']:.2f}s)\n")
                for det in log['detections']:
                    f.write(f"  - {det['class_cn']}: {det['direction_cn']} {det['distance']}m "
                           f"(置信度: {det['confidence']:.2f}, 危险等级: {det['danger_level']})\n")
                f.write("\n")

        print(f"检测日志: {log_path}")

    return output_path


In [ ]:

INPUT_VIDEO = "/Users/zyy/Downloads/e2eee24c8412776cd1517947ff225846.mp4"
MODEL_PATH = "yolo11x.pt"
CONFIDENCE = 0.35

# 单帧测试
test_frame, test_detections = test_random_frame(
    INPUT_VIDEO,
    model_path=MODEL_PATH,
    confidence=CONFIDENCE,
    use_chinese=True
)

In [54]:
output_video = process_video_with_progress(
    input_path=INPUT_VIDEO,
    model_path=MODEL_PATH,
    confidence=CONFIDENCE,
    use_chinese=True,
    show_info=True,
    save_detection_log=True
)

处理进度: 100%|█████████████████████████████████████| 4680/4680 [18:19<00:00,  4.26帧/s, 检测对象=0]


✅ 处理完成!
📁 输出文件: /Users/zyy/Downloads/e2eee24c8412776cd1517947ff225846_detected.mp4
📄 检测日志: /Users/zyy/Downloads/e2eee24c8412776cd1517947ff225846_detected.txt
